In [1]:
import pandas as pd
import datetime
df = pd.read_csv(r'order_brush_order.csv')
df.shape

FileNotFoundError: [Errno 2] File order_brush_order.csv does not exist: 'order_brush_order.csv'

In [ ]:
df.head(3)

In [ ]:
# df[df['userid'] == 196962305]
df['event_time'] = df['event_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
#pd.to_datetime(df['event_time'], format = '%Y-%m%d %H:%M:%S', errors = 'ignore')

In [ ]:
shop = df.groupby('shopid', as_index = False).apply(lambda x: x.sort_values('event_time'))
shop.head()

In [ ]:
#shop['user_count'] = shop.groupby(['shopid', 'userid'])['userid'].transform('count')
#suspect = shop[shop['user_count'] >= 3]
#suspect = suspect.sort_values(['shopid', 'userid'])
#suspect

In [ ]:
from collections import defaultdict
from datetime import timedelta
d1 = defaultdict(list)
for name, group in shop.groupby('shopid'):
    #print(name)
    order_count = len(group)
    brushing = False
    for idx in range(order_count - 2):
        time_limit = group['event_time'].iloc[idx] + timedelta(seconds = 3601)
        orders_1hr = group[group['event_time'] <= time_limit]
        unique_users = len(group['userid'].unique().tolist())
        orders = group.shape[0]
        concentrate_rate = orders/unique_users
        if concentrate_rate >= 3:
            # suspicious shops
            group['user_count'] = group.groupby('userid')['userid'].transform('count')
            group = group.sort_values('user_count')
            print(group)
            max_user = group[group['userid'] == max(group['user_count'])]['userid'].unique()
            d1[name].append(max_user)
    #if brushing:
        #d1[name[0]].append(str(name[1]))
#for key in d1.keys():
    #d1[key] = '&'.join(d1[key])
    #else:
    #    d1[name[0]].append(0)
d1
#d1[10159]

In [ ]:
new = pd.DataFrame(columns = ['shopid', 'userid'])
new['shopid'] = df['shopid'].unique()

In [ ]:
'''for idx in range(new.shape[0]):
    print(idx, new['shopid'].iloc[idx])
    if new['shopid'].iloc[idx] in d1:
        new['userid'].iloc[idx] = d1[idx]
    else:
        new['userid'].iloc[idx] = 0'''
new = pd.DataFrame(list(d1.items()), columns = ['shopid', 'userid'])
new
#new = new.replace({'[]':0})

In [ ]:
sub = pd.merge(new, df[['shopid']].drop_duplicates(), on = 'shopid', how = 'right')
ans = sub.fillna(0)
#ans[ans['userid'] == '[]']

In [ ]:
ans.to_csv(r'order_submission2.csv', index = False)

In [ ]:
ans.shape[0]

In [ ]:
df['total_orders'] = df.groupby('shopid')['shopid'].transform('count')

In [ ]:
df['total_users'] = df.groupby(['shopid', 'userid'])['shopid'].transform('count')
df['concentrate_rate'] = df['total_orders']/df['total_users']
df